In [43]:
import pandas as pd

df = pd.read_excel('/Users/khanhvx/Downloads/sndata.xlsx', sheet_name='sndata')

In [44]:
df.head()

,Date,account_name,account_id,campaign_name,campaign_id,region,cost,impression,clicks,views,Currency
0,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660032,AW-SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,1824669718717458,Ho Chi Minh City,5055936.0,598859,994,8803,VND
1,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,AW-SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,1824668610856049,Ho Chi Minh City,8180348.0,582048,939,11667,VND
2,2025-03-19,VNM_SN_NonGF_2025,7451448453630033921,AW-SN-Fino-FinoPromoSustain-1303-2103_TT_REA_A...,1826282585074737,Ho Chi Minh City,6877428.0,557469,935,8329,VND
3,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,AW-SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,1824669718717458,Hanoi,4511536.0,538414,1067,8265,VND
4,2025-04-30,VNM_SN_NonGF_2025,7451448453630033921,AW-SN-FM100-FM100PromoApr-May-1904-1705_TT_REA...,1829980160016481,Ho Chi Minh City,4579141.0,536644,773,6875,VND


In [45]:
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import sys
import sqlalchemy
from sqlalchemy import create_engine, Table, Column, Integer, Float, String, Text, Date, DateTime, Boolean, MetaData

def split_and_assign(df, column, column_type):
    # Define expected columns inside the function
    expected_columns_map = {
        "campaign_name": ['camp_code', 'platform', 'camp_type', 'buying_type', 'free_text1', 'year', 'optimization', 'buying_model', 'placement', 'freetext_2'],
        "adgroup_name": ['noneed_1', 'noneed_2', 'noneed_3', 'noneed_4', 'noneed_5', 'noneed_6', 'noneed_7', 'noneed_8', 'bidding_strategy', 'noneed_9'],
        "ad_name": ['creative_type', 'creative_format', 'creative_length', 'noneed_9', 'noneed_10', 'noneed_11', 'noneed_12', 'noneed_13', 'freetext_3'],
    }

    expected_columns = expected_columns_map.get(column_type, [])

    if column in df.columns:
        df[column] = df[column].fillna("").astype(str)
        split_cols = df[column].str.split('_', expand=True)

        # Pad with NaNs if needed
        num_actual_cols = split_cols.shape[1]
        num_expected_cols = len(expected_columns)
        if num_actual_cols < num_expected_cols:
            for i in range(num_actual_cols, num_expected_cols):
                split_cols[i] = pd.NA
        num_actual_cols = split_cols.shape[1]

        assigned_names = expected_columns[:num_actual_cols]
        extra_names = [f"extra_{i}" for i in range(num_actual_cols - len(assigned_names))]
        all_new_names = assigned_names + extra_names
        all_new_names = [f"{name}_split" if name in df.columns else name for name in all_new_names]

        split_cols.columns = all_new_names
        df = df.join(split_cols)

    # Split 'camp_code' into components
    if "camp_code" in df.columns:
        df["camp_code"] = df["camp_code"].fillna("").astype(str)
        campaign_split = df["camp_code"].str.split("-", expand=True)

        max_cols = 6
        campaign_split = campaign_split.iloc[:, :max_cols]
        for i in range(campaign_split.shape[1], max_cols):
            campaign_split[i] = np.nan

        campaign_split.columns = ["Funnel", "Category", "Brand Name", "Campaign Name", "Start Date", "End Date"]
        df = df.drop(columns=[col for col in campaign_split.columns if col in df.columns], errors="ignore")
        df = df.join(campaign_split)

    # Split 'free_text1' into components
    if "free_text1" in df.columns:
        df["free_text1"] = df["free_text1"].fillna("").astype(str)
        freetext_split = df["free_text1"].str.split("-", expand=True)

        max_cols = 6
        freetext_split = freetext_split.iloc[:, :max_cols]
        for i in range(freetext_split.shape[1], max_cols):
            freetext_split[i] = np.nan

        freetext_split.columns = ["Audience", "Gender", "Age", "Region", "Device", "Os"]
        df = df.drop(columns=[col for col in freetext_split.columns if col in df.columns], errors="ignore")
        df = df.join(freetext_split)

    # Create TA column
    if "Gender" in df.columns and "Age" in df.columns:
        df["Gender"] = df["Gender"].astype(str)
        df["Age"] = df["Age"].astype(str).str.zfill(4)
        df["TA"] = df["Gender"] + df["Age"].str[:2] + "-" + df["Age"].str[2:]

    # Format Start and End Dates
    if "Start Date" in df.columns and "End Date" in df.columns and "year" in df.columns:
        df["year"] = df["year"].astype(str)

        def format_date(row, col):
            if pd.notna(row[col]) and len(row[col]) == 4:
                return f"{row['year'][:2]}{row[col][:2]}-{row[col][2:]}"
            return None

        df["Start Date"] = df.apply(lambda row: format_date(row, "Start Date"), axis=1)
        df["End Date"] = df.apply(lambda row: format_date(row, "End Date"), axis=1)

        df["Start Date"] = pd.to_datetime(df["Start Date"], format="%Y-%m-%d", errors="coerce")
        df["End Date"] = pd.to_datetime(df["End Date"], format="%Y-%m-%d", errors="coerce")

    # Create ad_format column
    if all(col in df.columns for col in ["creative_type", "creative_format", "creative_length"]):
        df["ad_format"] = df["creative_type"].astype(str) + "_" + df["creative_format"].astype(str) + "_" + df["creative_length"].astype(str)

    # Drop unnecessary columns
    df = df.drop(columns=[col for col in df.columns if "noneed" in col], errors="ignore")

    return df


def upload_to_mysql(df, table_name, engine, if_exists='append'):
    """
    Uploads a DataFrame to a MySQL table with correct column types based on the DataFrame.

    :param df: Pandas DataFrame to upload
    :param table_name: Name of the table in the database
    :param engine: SQLAlchemy engine object
    :param if_exists: What to do if the table already exists ('fail', 'replace', 'append')
    """
    try:
        with engine.connect() as conn:
            metadata = MetaData()

            # Define table schema based on DataFrame's column types
            columns = []

            for col in df.columns:
                dtype = df[col].dtype
                
                # Map Pandas dtype to MySQL/SQLAlchemy types
                if pd.api.types.is_integer_dtype(dtype):
                    col_type = Integer
                elif pd.api.types.is_float_dtype(dtype):
                    col_type = Float
                elif pd.api.types.is_datetime64_any_dtype(dtype):
                    col_type = DateTime
                elif pd.api.types.is_bool_dtype(dtype):
                    col_type = Boolean
                elif pd.api.types.is_string_dtype(dtype):
                    col_type = Text(collation="utf8mb4_unicode_ci")  # UTF-8 support
                else:
                    col_type = String(255, collation="utf8mb4_unicode_ci")  # Default fallback

                columns.append(Column(col, col_type))

            table = Table(table_name, metadata, *columns, extend_existing=True)

            # Create the table with utf8mb4 encoding
            metadata.create_all(conn)

        # Convert object columns to string before uploading
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].astype(str)

        # Upload DataFrame to MySQL with proper type mapping
        dtype_mapping = {
            col: (
                sqlalchemy.types.Integer if pd.api.types.is_integer_dtype(df[col]) else
                sqlalchemy.types.Float if pd.api.types.is_float_dtype(df[col]) else
                sqlalchemy.types.DateTime if pd.api.types.is_datetime64_any_dtype(df[col]) else
                sqlalchemy.types.Boolean if pd.api.types.is_bool_dtype(df[col]) else
                sqlalchemy.types.Text(collation="utf8mb4_unicode_ci") if pd.api.types.is_string_dtype(df[col]) else
                sqlalchemy.types.String(255, collation="utf8mb4_unicode_ci")
            )
            for col in df.columns
        }

        df.to_sql(table_name, engine, if_exists=if_exists, index=False, dtype=dtype_mapping)

        print(f"✅ Data successfully uploaded to `{table_name}` with correct column types!")

    except Exception as e:
        print(f"❌ Error uploading data to `{table_name}`: {e}")


In [46]:
def mapping_naming(df):
    mapping_creative_type = {
        "VID": "Video",
        "IMAGE": "Image",
        "BAN": "Banner",
        "AUD": "Audio",
        "TEXT": "Text",
        "DISC": "Discovery Ad",
        "RESAD": "Responsive Ad",
        "ST": "Story",
        "GIF": "Gif",
        "CARO": "Carousel",
        "REEL": "Reel",
        "MULT": "Multiple",
        "DNM": "dynamic",
        "CLT": "collec",
        "TVC": "tvc",
        "SMS": "SMS",
        "LIVESTREAM": "livetream"
    }
    mapping_campaign_type = {
        "VRC": "View Reach Campaign",
        "VVC": "Video View Campaign",
        "PMAX": "PMAX",
        "DG": "DemandGen",
        "SEM": "SEM",
        "VAC": "Video Action Campaign",
        "REA": "Reach",
        "ENG": "Engage",
        "TRF": "Traffic",
        "LEAD": "Lead",
        "THRU": "Thruplay",
        "PDIS": "Programatic Display",
        "PVID": "Programatic Video",
        "OTT": "Programatic OTT",
        "MH": "Masthead",
        "APP": "App",
        "GDN": "GDN",
        "IS": "InStream",
        "BUMP": "Bumper",
        "TVN": "Trueview Nonskip",
        "LSA": "Livestream Ads",
        "CT": "Community Interaction",
        "PSA": "Product Shopping Ads",
        "VSA": "Video Shopping Ads"
    }
    mapping_kpi_metric = {
    "Imp": "Impression",
    "Click": "Click",
    "View": "View",
    "Reach": "Reach",
    "Engagement": "Engagement",
    "Lead Form": "Lead Form",
    "Message": "Message",
    "AppInstall": "App Install",
    "AppEngage": "App Engage"
}
    mapping_platform = {
        "DV360": "DV360",
        "DV360-DISP": "DV360 Display",
        "DV360-VID": "DV360 Video (non-YouTube)",
        "DV360-YT": "DV360 YouTube",
        "GG": "Google Ads",
        "GA-YT": "Google Ads YouTube",
        "GDN": "Google Display Network",
        "D-YT": "YouTube Direct",
        "FB": "Facebook",
        "TT": "TikTok",
        "IG": "Instagram",
        "YT": "YouTube",
    }
    mapping_campaign_funnel = {
        "AW": "Awareness",
        "CSD": "Consideration",
        "CVR": "Conversion"
    }

    if 'camp_type' in df.columns:
        df['camp_type'] = df['camp_type'].map(mapping_campaign_type)
    elif 'Format' in df.columns:
        df['Format'] = df['Format'].map(mapping_campaign_type)

    # Map Funnel
    if 'Funnel' in df.columns:
        df["Funnel"] = df["Funnel"].map(mapping_campaign_funnel)
    # Map Funnel
    if 'KPI Metric' in df.columns:
        df["KPI Metric"] = df["KPI Metric"].map(mapping_kpi_metric)
    # Map Platform
    if 'platform' in df.columns:
        df["Platform"] = df["platform"].map(mapping_platform)
    elif 'Platform' in df.columns:
        df["Platform"] = df["Platform"].map(mapping_platform)
        # Map Funnel


In [47]:
df_map_camp = pd.read_excel('/Users/khanhvx/Downloads/map sn2.xlsx', sheet_name='campaign')
df_map_ad = pd.read_excel('/Users/khanhvx/Downloads/map sn2.xlsx', sheet_name='ad')
df_map_camp = df_map_camp.drop_duplicates(subset='campaign_id', keep='last')
df_map_ad = df_map_ad.drop_duplicates(subset='ad_id', keep='last')


# Merge campaign name
df = df.merge(df_map_camp[['campaign_id', 'campaign_name']], on='campaign_id', how='left')
df['campaign_name_y'] = df['campaign_name_y'].fillna(df['campaign_name_x'])
df = df.rename(columns={'campaign_name_y': 'campaign_name'}).drop(columns=['campaign_name_x'])

In [48]:
# List of valid regions
# Standardize certain region names
df["region"] = df["region"].replace({
    "Ho Chi Minh": "Ho Chi Minh City",
    "Ha Noi": "Hanoi"
})

valid_regions = [
    "An Giang", "Ba Ria – Vung Tau", "Bac Giang", "Bac Kan", "Bac Lieu", "Bac Ninh",
    "Ben Tre", "Binh Dinh", "Binh Duong", "Binh Phuoc", "Binh Thuan", "Ca Mau",
    "Can Tho", "Cao Bang", "Da Nang", "Dak Lak", "Dak Nong", "Dien Bien", "Dong Nai",
    "Dong Thap", "Gia Lai", "Ha Giang", "Ha Nam", "Ha Tinh", "Hai Duong", "Hai Phong",
    "Hanoi", "Hau Giang", "Ho Chi Minh City", "Hoa Binh", "Hung Yen", "Khanh Hoa",
    "Kien Giang", "Kon Tum", "Lai Chau", "Lam Dong", "Lang Son", "Lao Cai", "Long An",
    "Nam Dinh", "Nghe An", "Ninh Binh", "Ninh Thuan", "Phu Tho", "Phu Yen", "Quang Binh",
    "Quang Nam", "Quang Ngai", "Quang Ninh", "Quang Tri", "Soc Trang", "Son La",
    "Tay Ninh", "Thai Binh", "Thai Nguyen", "Thanh Hoa", "Thua Thien Hue", "Tien Giang",
    "Tra Vinh", "Tuyen Quang", "Unknown", "Vinh Long", "Vinh Phuc", "Yen Bai"
]

# Filter the DataFrame to keep only valid regions
df = df[df["region"].isin(valid_regions)]

In [49]:

import re
campaign_name_expected_columns = ["Funnel", 'camp_code', 'platform', 'camp_type', 'buying_type', 'free_text1', 'year', 'KPI Metric',
                                  'buying_model', 'placement', 'Campaign phase', 'freetext_2']

adgroup_name_expected_columns = ['noneed_1', 'noneed_2', 'noneed_3', 'noneed_4', 'noneed_5', 'noneed_6', 'noneed_7',
                                 'noneed_8', 'bidding_strategy', 'noneed_9']

# Original list with 9 names
ad_name_expected_columns = ['creative_type', 'creative_format', 'creative_length', 'noneed_9', 'noneed_10', 'noneed_11', 'noneed_12',
                            'noneed_13', 'freetext_3']

def split_and_assign(df, column, expected_columns):
    if column in df.columns:
        df[column] = df[column].fillna("").astype(str)
        split_cols = df[column].str.split('_', expand=True)

        num_expected = len(expected_columns)
        num_actual = split_cols.shape[1]

        # Determine how many columns to actually use and name (max is num_expected)
        cols_to_assign = min(num_actual, num_expected)

        # Take only the relevant columns from the split (up to num_expected)
        split_cols_final = split_cols.iloc[:, :cols_to_assign]

        # Assign the corresponding expected names
        split_cols_final.columns = expected_columns[:cols_to_assign]

        # Check for existing columns before joining to avoid duplicate column errors
        cols_to_drop_before_join = [col for col in split_cols_final.columns if col in df.columns]
        df = df.drop(columns=cols_to_drop_before_join, errors='ignore')

        df = df.join(split_cols_final)


    # ✅ Improved camp_code splitting (keep this logic as is)
    if "camp_code" in df.columns:
        # ... (your existing camp_code splitting logic) ...
        df["camp_code"] = df["camp_code"].fillna("").astype(str)
        # First, split from the right on last two hyphens → Start Date and End Date
        campaign_split = df["camp_code"].str.rsplit("-", n=2, expand=True)
        campaign_split_final = pd.DataFrame() # Initialize empty DataFrame

        if campaign_split.shape[1] == 3:
            prefix_part = campaign_split[0]
            start_date_str = campaign_split[1]
            end_date_str = campaign_split[2]

            # Now, split the prefix part into the first 4 components
            prefix_split = prefix_part.str.split("-", n=2, expand=True)
            # Ensure prefix_split has 4 columns, padding if necessary
            for i in range(prefix_split.shape[1], 3):
                 prefix_split[i] = "" # Add empty columns if split is short
            prefix_split.columns = ["Category", "Brand Name", "Campaign Name"]


            # Combine all parts
            campaign_split_final = pd.concat([prefix_split, start_date_str, end_date_str], axis=1)
            campaign_split_final.columns = ["Category", "Brand Name", "Campaign Name", "Plan Start Date", "Plan End Date"]

             # Drop existing columns before joining to avoid conflict
            df = df.drop(columns=[col for col in campaign_split_final.columns if col in df.columns], errors="ignore")
            df = df.join(campaign_split_final)
        # else: handle cases where camp_code doesn't split into 3 parts if needed


    # ✅ free_text1 splitting (keep this logic as is)
    if "free_text1" in df.columns:
        # ... (your existing free_text1 splitting logic) ...
        df["free_text1"] = df["free_text1"].fillna("").astype(str)
        freetext_split = df["free_text1"].str.split("-", expand=True)

        max_cols = 6
        # Select up to max_cols, don't error if fewer exist
        freetext_split = freetext_split.iloc[:, :min(freetext_split.shape[1], max_cols)]

        # Pad with NaN if fewer than max_cols resulted from the split
        for i in range(freetext_split.shape[1], max_cols):
            freetext_split[i] = np.nan # Use standard column names

        # Assign standard names
        freetext_split.columns = ["Audience group", "Gender", "Age", "Region", "Device", "Os"][:freetext_split.shape[1]] # Only assign names for existing columns

        df = df.drop(columns=[col for col in freetext_split.columns if col in df.columns], errors="ignore")
        df = df.join(freetext_split)


    # ✅ Target Audience creation (keep this logic as is)
    if "Gender" in df.columns and "Age" in df.columns:
         # Ensure columns exist before proceeding
        df["Gender"] = df["Gender"].fillna("NA").astype(str) # Handle potential NaNs from split
        df["Age"] = df["Age"].fillna("0000").astype(str).str.zfill(4) # Handle NaNs and ensure padding
        # Use .str accessor safely
        df["TA"] = df["Gender"] + df["Age"].str[:2] + "-" + df["Age"].str[2:]


    # ✅ Date formatting
    # Ensure 'year' column exists and handle potential errors before using iloc[0]
    current_year = datetime.now().year # Default year if 'year' column is missing/empty
    if "year" in df.columns and not df["year"].isnull().all():
        # Try converting the first valid year value to int
        try:
            first_valid_year_index = df['year'].first_valid_index()
            if first_valid_year_index is not None:
                current_year = int(df['year'].loc[first_valid_year_index])
            else:
                 print("Warning: 'year' column has no valid values. Using current system year.")
        except (ValueError, TypeError):
            print(f"Warning: Could not convert value in 'year' column to integer. Using current system year: {current_year}")


    if "Plan Start Date" in df.columns and "Plan End Date" in df.columns:
        def parse_ddmm(val, year_to_use):
            if pd.isna(val) or val == '':
                return pd.NaT
            try:
                # Handle potential float inputs from splits
                val_str = str(val).split('.')[0] if isinstance(val, float) else str(val)
                return datetime.strptime(val_str, "%d%m").replace(year=year_to_use)
            except (ValueError, TypeError):
                 # Handle cases like 'NaT', '', invalid formats
                return pd.NaT

        # Apply using the determined year
        df["Plan Start Date"] = df["Plan Start Date"].apply(lambda x: parse_ddmm(x, current_year))
        df["Plan End Date"] = df["Plan End Date"].apply(lambda x: parse_ddmm(x, current_year))


    # ✅ Combine creative fields (keep this logic as is)
    if all(col in df.columns for col in ["creative_type", "creative_format", "creative_length"]):
        df["ad_format"] = (df["creative_type"].fillna("").astype(str) + "_" +
                           df["creative_format"].fillna("").astype(str) + "_" +
                           df["creative_length"].fillna("").astype(str))

    # ✅ Handle freetext_2 as Actual Start and End Dates (from the last date pattern in the text)
    if "freetext_2" in df.columns:
        df["freetext_2"] = df["freetext_2"].fillna("").astype(str)

        def parse_freetext2(val, year_to_use):
            # Find all matches of the date pattern in the text
            matches = re.findall(r"(\d{4})-(\d{4})", val)
            if not matches:
                return pd.NaT, pd.NaT

            # Use the last match
            start_str, end_str = matches[-1]

            try:
                start_date = datetime.strptime(start_str, "%d%m").replace(year=year_to_use)
            except ValueError:
                start_date = pd.NaT
            try:
                end_date = datetime.strptime(end_str, "%d%m").replace(year=year_to_use)
            except ValueError:
                end_date = pd.NaT

            return start_date, end_date

        actual_dates = df["freetext_2"].apply(lambda x: pd.Series(parse_freetext2(x, current_year)))
        actual_dates.columns = ["Actual Start Date", "Actual End Date"]

        df = df.drop(columns=[col for col in actual_dates.columns if col in df.columns], errors="ignore")
        df = df.join(actual_dates)

    # ✅ Drop unnecessary columns (keep this logic as is)
    df = df.drop(columns=[col for col in df.columns if "noneed" in col], errors="ignore")

    return df


# ... (keep your rename and process_campaign_data function) ...

def process_campaign_data(df):
    """Processes a DataFrame by filtering campaigns, splitting columns, and formatting TA & dates."""

    # Always start with a copy to avoid modifying the original DataFrame
    df_filtered = df.copy()
    # Informational warning if 'campaign_name' is not found
    if 'campaign_name' not in df.columns:
        print("Warning: 'campaign_name' column not found. Skipping campaign_name splitting.")
    else:
        # Apply splitting logic to 'campaign_name'
        df_filtered = split_and_assign(df_filtered, 'campaign_name', campaign_name_expected_columns)

    # *** Check if 'ad_group' or 'ad_group_name' exists after renames ***
    if 'ad_group_name' in df_filtered.columns:
        df_filtered = split_and_assign(df_filtered, 'ad_group_name', adgroup_name_expected_columns)
    elif 'ad_group' in df_filtered.columns:
        print("Warning: Processing 'ad_group' instead of 'ad_group_name'")
        df_filtered = split_and_assign(df_filtered, 'ad_group', adgroup_name_expected_columns)

    # *** Check if 'ad_name' exists after renames ***
    if 'ad_name' in df_filtered.columns:
        df_filtered = split_and_assign(df_filtered, 'ad_name', ad_name_expected_columns)

    return df_filtered

df_sn = process_campaign_data(df)
df_sn

,Date,account_name,account_id,campaign_id,region,cost,impression,clicks,views,Currency,...,Plan End Date,Audience group,Gender,Age,Region,Device,Os,TA,Actual Start Date,Actual End Date
0,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660032,1824669718717458,Ho Chi Minh City,5055936.0,598859,994,8803,VND,...,2025-03-29,YoungWomen,F,1824,35C,AllDV,AllOS,F18-24,2025-02-22,2025-03-29
1,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824668610856049,Ho Chi Minh City,8180348.0,582048,939,11667,VND,...,2025-03-29,Mom,F,2544,35C,AllDV,AllOS,F25-44,2025-02-22,2025-03-29
2,2025-03-19,VNM_SN_NonGF_2025,7451448453630033921,1826282585074737,Ho Chi Minh City,6877428.0,557469,935,8329,VND,...,2025-03-31,Group1 (focus),F,2544,32C,AllDV,AllOS,F25-44,2025-03-13,2025-03-21
3,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824669718717458,Hanoi,4511536.0,538414,1067,8265,VND,...,2025-03-29,YoungWomen,F,1824,35C,AllDV,AllOS,F18-24,2025-02-22,2025-03-29
4,2025-04-30,VNM_SN_NonGF_2025,7451448453630033921,1829980160016481,Ho Chi Minh City,4579141.0,536644,773,6875,VND,...,2025-05-31,Group1 ExcludeGroup3,F,2544,NAT,AllDV,AllOS,F25-44,2025-04-19,2025-05-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146546,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Long,0.0,5,0,0,VND,...,2025-06-30,MASS,P,18plus,NAT,AllDV,AllOS,P18-plus,NaT,NaT
146547,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Phuc,0.0,1,0,0,VND,...,2025-06-30,MASS,P,18plus,NAT,AllDV,AllOS,P18-plus,NaT,NaT
146548,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ba Ria – Vung Tau,0.0,3,0,0,VND,...,2025-06-30,MASS,P,18plus,NAT,AllDV,AllOS,P18-plus,NaT,NaT
146549,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ben Tre,6990.0,1,1,0,VND,...,2025-06-30,MASS,P,18plus,NAT,AllDV,AllOS,P18-plus,NaT,NaT


In [50]:
df_sn
# Format df_sn column headers
df_sn.columns = [
    col.replace('_', ' ').title().replace(' ', '') if col != 'Brand_Name' else 'Brand'
    for col in df_sn.columns
]

df_sn

,Date,AccountName,AccountId,CampaignId,Region,Cost,Impression,Clicks,Views,Currency,...,PlanEndDate,AudienceGroup,Gender,Age,Region,Device,Os,Ta,ActualStartDate,ActualEndDate
0,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660032,1824669718717458,Ho Chi Minh City,5055936.0,598859,994,8803,VND,...,2025-03-29,YoungWomen,F,1824,35C,AllDV,AllOS,F18-24,2025-02-22,2025-03-29
1,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824668610856049,Ho Chi Minh City,8180348.0,582048,939,11667,VND,...,2025-03-29,Mom,F,2544,35C,AllDV,AllOS,F25-44,2025-02-22,2025-03-29
2,2025-03-19,VNM_SN_NonGF_2025,7451448453630033921,1826282585074737,Ho Chi Minh City,6877428.0,557469,935,8329,VND,...,2025-03-31,Group1 (focus),F,2544,32C,AllDV,AllOS,F25-44,2025-03-13,2025-03-21
3,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824669718717458,Hanoi,4511536.0,538414,1067,8265,VND,...,2025-03-29,YoungWomen,F,1824,35C,AllDV,AllOS,F18-24,2025-02-22,2025-03-29
4,2025-04-30,VNM_SN_NonGF_2025,7451448453630033921,1829980160016481,Ho Chi Minh City,4579141.0,536644,773,6875,VND,...,2025-05-31,Group1 ExcludeGroup3,F,2544,NAT,AllDV,AllOS,F25-44,2025-04-19,2025-05-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146546,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Long,0.0,5,0,0,VND,...,2025-06-30,MASS,P,18plus,NAT,AllDV,AllOS,P18-plus,NaT,NaT
146547,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Phuc,0.0,1,0,0,VND,...,2025-06-30,MASS,P,18plus,NAT,AllDV,AllOS,P18-plus,NaT,NaT
146548,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ba Ria – Vung Tau,0.0,3,0,0,VND,...,2025-06-30,MASS,P,18plus,NAT,AllDV,AllOS,P18-plus,NaT,NaT
146549,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ben Tre,6990.0,1,1,0,VND,...,2025-06-30,MASS,P,18plus,NAT,AllDV,AllOS,P18-plus,NaT,NaT


In [51]:
mapping_naming(df_sn)


In [52]:
df_sn.columns

Index(['Date', 'AccountName', 'AccountId', 'CampaignId', 'Region', 'Cost',
       'Impression', 'Clicks', 'Views', 'Currency', 'CampaignName', 'Funnel',
       'CampCode', 'Platform', 'CampType', 'BuyingType', 'FreeText1', 'Year',
       'KpiMetric', 'BuyingModel', 'Placement', 'CampaignPhase', 'Freetext2',
       'Category', 'BrandName', 'CampaignName', 'PlanStartDate', 'PlanEndDate',
       'AudienceGroup', 'Gender', 'Age', 'Region', 'Device', 'Os', 'Ta',
       'ActualStartDate', 'ActualEndDate'],
      dtype='object')

In [54]:
def calculate_cost(row):
    if row['Currency'] == 'USD':
        return row['Cost'] * 26000
    elif row['Currency'] == 'VND':
        return row['Cost'] * 1  # No exchange rate needed for VND
    else:
        return row['Cost']  # For other currencies, just return the cost in USD
df_sn['Cost'] = df_sn.apply(calculate_cost, axis=1)

In [ ]:
df_sn

In [ ]:
df_sn.drop(columns=['cost', 'Campaign Name'], inplace=True)

In [ ]:
df_sn.columns = df_sn.columns.str.strip().str.replace(r'\W+', '_', regex=True)

In [ ]:
df_sn['campaign_id'] = df_sn['campaign_id'].astype(str)

In [ ]:
df_sn['Audience'] = df_sn['Audience_group'].astype(str) + '\n' + df_sn['TA'].astype(str)
df_sn

,Date,account_name,account_id,campaign_id,region,impression,clicks,views,Currency,campaign_name,...,Gender,Age,Device,Os,TA,Actual_Start_Date,Actual_End_Date,Platform,Cost,Audience
0,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660032,1824669718717458,Ho Chi Minh City,598859,994,8803,VND,AW_SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,...,F,1824,AllDV,AllOS,F18-24,2025-02-22,2025-03-29,TikTok,5055936.0,YoungWomen\nF18-24
1,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824668610856049,Ho Chi Minh City,582048,939,11667,VND,AW_SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,...,F,2544,AllDV,AllOS,F25-44,2025-02-22,2025-03-29,TikTok,8180348.0,Mom\nF25-44
2,2025-03-19,VNM_SN_NonGF_2025,7451448453630033921,1826282585074737,Ho Chi Minh City,557469,935,8329,VND,AW_SN-Fino-FinoPromoSustain-1203-3103_TT_REA_A...,...,F,2544,AllDV,AllOS,F25-44,2025-03-13,2025-03-21,TikTok,6877428.0,Group1 (focus)\nF25-44
3,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824669718717458,Hanoi,538414,1067,8265,VND,AW_SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,...,F,1824,AllDV,AllOS,F18-24,2025-02-22,2025-03-29,TikTok,4511536.0,YoungWomen\nF18-24
4,2025-04-30,VNM_SN_NonGF_2025,7451448453630033921,1829980160016481,Ho Chi Minh City,536644,773,6875,VND,AW_SN-FM100-FM100PromoAprMay-2403-3105_TT_REA_...,...,F,2544,AllDV,AllOS,F25-44,2025-04-19,2025-05-17,TikTok,4579141.0,Group1 ExcludeGroup3 \nF25-44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146546,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Long,5,0,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,P,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,0.0,MASS\nP18-plus
146547,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Phuc,1,0,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,P,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,0.0,MASS\nP18-plus
146548,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ba Ria – Vung Tau,3,0,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,P,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,0.0,MASS\nP18-plus
146549,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ben Tre,1,1,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,P,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,6990.0,MASS\nP18-plus


In [ ]:
region_to_code = {
    "An Giang": 89,
    "Bac Giang": 24,
    "Bac Kan": 6,
    "Bac Lieu": 95,
    "Bac Ninh": 27,
    "Ben Tre": 83,
    "Binh Dinh": 52,
    "Binh Duong": 74,
    "Binh Phuoc": 70,
    "Binh Thuan": 60,
    "Ca Mau": 96,
    "Can Tho": 92,
    "Cao Bang": 4,
    "Dak Lak": 66,
    "Dak Nong": 67,
    "Dien Bien": 11,
    "Dong Nai": 75,
    "Dong Thap": 87,
    "Gia Lai": 64,
    "Ha Giang": 2,
    "Ha Nam": 35,
    "Hanoi": 1,
    "Ha Tinh": 42,
    "Hai Duong": 30,
    "Hai Phong": 31,
    "Hau Giang": 93,
    "Hoa Binh": 17,
    "Hung Yen": 33,
    "Kon Tum": 62,
    "Lai Chau": 12,
    "Lam Dong": 68,
    "Lang Son": 20,
    "Lao Cai": 10,
    "Long An": 80,
    "Nam Dinh": 36,
    "Nghe An": 40,
    "Ninh Binh": 37,
    "Ninh Thuan": 58,
    "Phu Tho": 25,
    "Phu Yen": 54,
    "Quang Binh": 44,
    "Quang Nam": 49,
    "Quang Ngai": 51,
    "Quang Ninh": 22,
    "Quang Tri": 45,
    "Soc Trang": 94,
    "Son La": 14,
    "Tay Ninh": 72,
    "Thai Binh": 34,
    "Thai Nguyen": 19,
    "Thanh Hoa": 38,
    "Thua Thien - Hue": 46,
    "Tien Giang": 82,
    "Ho Chi Minh City": 79,
    "Tra Vinh": 84,
    "Tuyen Quang": 8,
    "Vinh Long": 86,
    "Vinh Phuc": 26,
    "Yen Bai": 15,
    "Da Nang": 48,
    "Khanh Hoa": 56,
    "Kien Giang": 91,
    "Ba Ria - Vung Tau": 77
}
df_sn["Code"] = df_sn["region"].map(region_to_code)
df_sn

,Date,account_name,account_id,campaign_id,region,impression,clicks,views,Currency,campaign_name,...,Age,Device,Os,TA,Actual_Start_Date,Actual_End_Date,Platform,Cost,Audience,Code
0,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660032,1824669718717458,Ho Chi Minh City,598859,994,8803,VND,AW_SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,...,1824,AllDV,AllOS,F18-24,2025-02-22,2025-03-29,TikTok,5055936.0,YoungWomen\nF18-24,79.0
1,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824668610856049,Ho Chi Minh City,582048,939,11667,VND,AW_SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,...,2544,AllDV,AllOS,F25-44,2025-02-22,2025-03-29,TikTok,8180348.0,Mom\nF25-44,79.0
2,2025-03-19,VNM_SN_NonGF_2025,7451448453630033921,1826282585074737,Ho Chi Minh City,557469,935,8329,VND,AW_SN-Fino-FinoPromoSustain-1203-3103_TT_REA_A...,...,2544,AllDV,AllOS,F25-44,2025-03-13,2025-03-21,TikTok,6877428.0,Group1 (focus)\nF25-44,79.0
3,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824669718717458,Hanoi,538414,1067,8265,VND,AW_SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,...,1824,AllDV,AllOS,F18-24,2025-02-22,2025-03-29,TikTok,4511536.0,YoungWomen\nF18-24,1.0
4,2025-04-30,VNM_SN_NonGF_2025,7451448453630033921,1829980160016481,Ho Chi Minh City,536644,773,6875,VND,AW_SN-FM100-FM100PromoAprMay-2403-3105_TT_REA_...,...,2544,AllDV,AllOS,F25-44,2025-04-19,2025-05-17,TikTok,4579141.0,Group1 ExcludeGroup3 \nF25-44,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146546,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Long,5,0,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,0.0,MASS\nP18-plus,86.0
146547,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Phuc,1,0,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,0.0,MASS\nP18-plus,26.0
146548,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ba Ria – Vung Tau,3,0,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,0.0,MASS\nP18-plus,NaN
146549,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ben Tre,1,1,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,6990.0,MASS\nP18-plus,83.0


In [ ]:
df_sn.rename(columns={"Date": "report_date"}, inplace=True)

In [ ]:
df_sn

,report_date,account_name,account_id,campaign_id,region,impression,clicks,views,Currency,campaign_name,...,Age,Device,Os,TA,Actual_Start_Date,Actual_End_Date,Platform,Cost,Audience,Code
0,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660032,1824669718717458,Ho Chi Minh City,598859,994,8803,VND,AW_SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,...,1824,AllDV,AllOS,F18-24,2025-02-22,2025-03-29,TikTok,5055936.0,YoungWomen\nF18-24,79.0
1,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824668610856049,Ho Chi Minh City,582048,939,11667,VND,AW_SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,...,2544,AllDV,AllOS,F25-44,2025-02-22,2025-03-29,TikTok,8180348.0,Mom\nF25-44,79.0
2,2025-03-19,VNM_SN_NonGF_2025,7451448453630033921,1826282585074737,Ho Chi Minh City,557469,935,8329,VND,AW_SN-Fino-FinoPromoSustain-1203-3103_TT_REA_A...,...,2544,AllDV,AllOS,F25-44,2025-03-13,2025-03-21,TikTok,6877428.0,Group1 (focus)\nF25-44,79.0
3,2025-03-31,VNM_SN_Green Farm_2025,7451535237055660033,1824669718717458,Hanoi,538414,1067,8265,VND,AW_SN-GreenFarm-GreenFarmCoreSustainFeb-2202-2...,...,1824,AllDV,AllOS,F18-24,2025-02-22,2025-03-29,TikTok,4511536.0,YoungWomen\nF18-24,1.0
4,2025-04-30,VNM_SN_NonGF_2025,7451448453630033921,1829980160016481,Ho Chi Minh City,536644,773,6875,VND,AW_SN-FM100-FM100PromoAprMay-2403-3105_TT_REA_...,...,2544,AllDV,AllOS,F25-44,2025-04-19,2025-05-17,TikTok,4579141.0,Group1 ExcludeGroup3 \nF25-44,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146546,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Long,5,0,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,0.0,MASS\nP18-plus,86.0
146547,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22573221049,Vinh Phuc,1,0,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,0.0,MASS\nP18-plus,26.0
146548,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ba Ria – Vung Tau,3,0,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,0.0,MASS\nP18-plus,NaN
146549,2025-06-15,VNM_SBTE_AllBrands_2025_VND,732-640-2299,22563244767,Ben Tre,1,1,0,VND,CSD_SBTE-DielacGold-SEMAWO-0105-3006_GG_SEM_AU...,...,18plus,AllDV,AllOS,P18-plus,NaT,NaT,Google Ads,6990.0,MASS\nP18-plus,83.0


In [ ]:
from sqlalchemy import create_engine
import pandas as pd

DB_TYPE = 'mysql' 
DB_HOST = '125.212.245.36'
DB_PORT = '3306'  
DB_USER = 'dashboard_report_vinamilk'
DB_PASS = '6vtRhNxa79zPsfs5'
DB_NAME = 'dashboard_report_vinamilk'

try:
    conn = mysql.connector.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME,
        charset="utf8mb4"
    )
    
    if conn.is_connected():
        cursor = conn.cursor()
        print("Connected to MySQL successfully!")
    else:
        print("Failed to connect.")

except mysql.connector.Error as e:
    print(f"Error: {e}")

cursor = conn.cursor()
cursor.execute("SET NAMES utf8mb4;") 
cursor.execute("SET CHARACTER SET utf8mb4;")
engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

def upload_to_mysql(df, table_name, engine, if_exists='replace'):
    """
    Uploads a DataFrame to a MySQL table with correct column types based on the DataFrame.

    :param df: Pandas DataFrame to upload
    :param table_name: Name of the table in the database
    :param engine: SQLAlchemy engine object
    :param if_exists: What to do if the table already exists ('fail', 'replace', 'append')
    """
    try:
        with engine.connect() as conn:
            metadata = MetaData()

            # Define table schema based on DataFrame's column types
            columns = []

            for col in df.columns:
                dtype = df[col].dtype
                
                # Map Pandas dtype to MySQL/SQLAlchemy types
                if pd.api.types.is_integer_dtype(dtype):
                    col_type = Integer
                elif pd.api.types.is_float_dtype(dtype):
                    col_type = Float
                elif pd.api.types.is_datetime64_any_dtype(dtype):
                    col_type = DateTime
                elif pd.api.types.is_bool_dtype(dtype):
                    col_type = Boolean
                elif pd.api.types.is_string_dtype(dtype):
                    col_type = Text(collation="utf8mb4_unicode_ci")  # UTF-8 support
                else:
                    col_type = String(255, collation="utf8mb4_unicode_ci")  # Default fallback

                columns.append(Column(col, col_type))

            table = Table(table_name, metadata, *columns, extend_existing=True)

            # Create the table with utf8mb4 encoding
            metadata.create_all(conn)

        # Convert object columns to string before uploading
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].astype(str)

        # Upload DataFrame to MySQL with proper type mapping
        dtype_mapping = {
            col: (
                sqlalchemy.types.Integer if pd.api.types.is_integer_dtype(df[col]) else
                sqlalchemy.types.Float if pd.api.types.is_float_dtype(df[col]) else
                sqlalchemy.types.DateTime if pd.api.types.is_datetime64_any_dtype(df[col]) else
                sqlalchemy.types.Boolean if pd.api.types.is_bool_dtype(df[col]) else
                sqlalchemy.types.Text(collation="utf8mb4_unicode_ci") if pd.api.types.is_string_dtype(df[col]) else
                sqlalchemy.types.String(255, collation="utf8mb4_unicode_ci")
            )
            for col in df.columns
        }

        df.to_sql(table_name, engine, if_exists=if_exists, index=False, dtype=dtype_mapping)

        print(f"✅ Data successfully uploaded to `{table_name}` with correct column types!")

    except Exception as e:
        print(f"❌ Error uploading data to `{table_name}`: {e}")

# Example usage:
# engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
upload_to_mysql(df_sn, 'report_campaign_region_api2', engine)

Connected to MySQL successfully!
✅ Data successfully uploaded to `report_campaign_region_api` with correct column types!
